In [22]:
import datetime

import cvxpy as cp

import numpy as np
import pandas as pd


import matplotlib.pyplot as plt

from haversine import haversine_vector, Unit

In [23]:
def temporal_clustering(H,demand):
    
    def str_to_dt(dt_str):
        return datetime.datetime.strptime(dt_str, '%H:%M:%S').time()

    
    # temporal clustering
    demand.insert(3,'start_timeslot', demand['start_time'].apply(lambda x:int(np.floor((60*str_to_dt(x).hour + str_to_dt(x).minute)/H))))
    demand.insert(6,'end_timeslot', demand['end_time'].apply(lambda x:int(np.floor((60*str_to_dt(x).hour + str_to_dt(x).minute)/H))))

In [24]:
def spatial_clustering(d,demand):
    stations = pd.read_csv('stations.csv',index_col=0)
    
    stn_coords = list(zip(stations.y, stations.x))

    D = haversine_vector(stn_coords, stn_coords, Unit.KILOMETERS, comb=True)
    R = 1*(D < d) # 1 km threshold
    
    # ilp to select cluster centers
    x = cp.Variable((len(stations),),boolean=True)
    prob = cp.Problem(cp.Minimize(cp.sum(x)), [x.T@R>=1])
    prob.solve()
    
    # map each station to a cluster
    stations['cc'] = x.value
    cluster_centers = stations.loc[stations.cc==1]
    cc_coords = list(zip(cluster_centers.x, cluster_centers.y))
    stations['cc_id'] = np.argmin(haversine_vector(cc_coords, stn_coords, Unit.KILOMETERS, comb=True), axis=1)
    
    station_to_cluster_map = dict(zip(stations.index, stations.cc_id))
    
    # spatial clustering
    demand['start_cluster'] = demand['start_station'].apply(lambda x: station_to_cluster_map[x])
    demand['end_cluster'] = demand['end_station'].apply(lambda x: station_to_cluster_map[x])
    
    return station_to_cluster_map, stations

In [25]:
cd C:\Work\WORK_PACKAGE\Demand_forecasting\github\blue-sg/Demand_forecasting/BLUESG_Demand_data/Data-preprocessing_data_generation

[WinError 3] The system cannot find the path specified: 'C:\\Work\\WORK_PACKAGE\\Demand_forecasting\\github\\blue-sg/Demand_forecasting/BLUESG_Demand_data/Data-preprocessing_data_generation'
c:\Work\WORK_PACKAGE\Demand_forecasting\BLUESG_Demand_data\Data-preprocessing_data_generation


In [26]:
ls

 Volume in drive C is Windows 
 Volume Serial Number is A2F9-40D5

 Directory of c:\Work\WORK_PACKAGE\Demand_forecasting\BLUESG_Demand_data\Data-preprocessing_data_generation

08/03/2023  11:05 am    <DIR>          .
08/03/2023  11:05 am    <DIR>          ..
31/01/2023  01:26 pm            15,968 Clstr_inflow_outflow_data_generator_new.ipynb
08/04/2022  02:31 pm        51,077,895 demand.csv
08/03/2023  11:04 am         1,781,831 inflow_clstr_dem.csv
08/03/2023  11:04 am         1,784,302 outflow_clstr_dem.csv
08/03/2023  11:04 am           123,686 station_cc_id.csv
08/03/2023  11:28 am            19,478 station_inflow_outflow_data_generator_new.ipynb
17/04/2022  08:54 pm           120,634 stations.csv
               7 File(s)     54,923,794 bytes
               2 Dir(s)  21,308,481,536 bytes free


In [27]:
if __name__ == "__main__":
    demand = pd.read_csv('demand.csv',index_col=0)

## start
# DATAFRAME T cluster level demand

    H,d = 10,0.01 # H:Timeslot duration in minutes, d = radius for clustering in km
    #temporal clustering
    temporal_clustering(H,demand)
    
    # #spatial clustering
    # station_to_cluster_map, stations = spatial_clustering(d,demand)
    
    # print(station_to_cluster_map)
    # #print(stations['region'])
    # stations.to_csv('station_cc_id_new.csv', index=False)

    #aggregated demand table
    T = pd.pivot_table(demand, values = 'car', index=["start_date","start_timeslot"], columns=["start_station"], aggfunc=lambda x: len(x.unique()))
    IN = pd.pivot_table(demand, values = 'car', index=["end_date","end_timeslot"], columns=["end_station"], aggfunc=lambda x: len(x.unique()))
    IN.fillna(0, inplace=True)
    T.fillna(0, inplace=True)
    
    # missing_clusters = [x for x in range(T.columns[-1]) if x not in T.columns]
    # missing_clusters.sort()

    # for ms in missing_clusters:
    #     T.insert(ms, ms, 0)

    print(T)

    T.to_csv('outflow_station_dem.csv', index=False)

    # missing_clusters = [x for x in range(IN.columns[-1]) if x not in IN.columns]
    # missing_clusters.sort()
    
    # for ms in missing_clusters:
    #     IN.insert(ms, ms, 0)

    # print(IN)
    IN.to_csv('inflow_station_dem.csv', index=False)
    
## stop

    #raw_data = T.to_numpy()



start_station              4    5    6    7    8    9    10   11   12   13   \
start_date start_timeslot                                                     
2021-09-23 130             0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
           135             0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
           137             0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
           138             0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
           139             0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
...                        ...  ...  ...  ...  ...  ...  ...  ...  ...  ...   
2021-12-23 110             0.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0   
           111             0.0  0.0  0.0  0.0  0.0  1.0  0.0  1.0  0.0  0.0   
           112             0.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0   
           113             0.0  0.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0   
           114             0.0  0.0  0.0  0.0  0.0  